# Evaluate feature spaces

Perform two analyses to evaluate four different feature spaces.

The two analyses are:

1. UMAP
2. Clustering with Silhouette and enrichment evaluations

The feature spaces include:

1. All features
2. Feature selected features by traditional methods
3. Bortezomib signature features
4. All features except bortezomib signature features

In [1]:
import umap
import pathlib
import random
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from scipy.stats import fisher_exact

from typing import List, Union

from pycytominer.cyto_utils import infer_cp_features

/Users/waygr/miniforge3/envs/resistance-mechanisms-signature/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/waygr/miniforge3/envs/resistance-mechanisms-signature/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/waygr/minif

In [2]:
np.random.seed(1234)

In [3]:
def process_umap(
        data_df: pd.DataFrame,
        umap_category: str,
        features: Union[str, List[str]] = "infer"
        ) -> pd.DataFrame:
    """
    Apply UMAP to a dataframe and return a dataframe with UMAP coordinates and metadata.

    Parameters
    ----------
    data_df : pd.DataFrame
        The input dataframe. Should include feature columns and metadata columns.
    umap_category : str
        A label to add to the output dataframe under the column 'Metadata_umap_category'.
    features : str or list of str, default 'infer'
        The names of the feature columns to use for UMAP. If 'infer', all non-metadata columns are used.

    Returns
    -------
    pd.DataFrame
        A dataframe with the UMAP coordinates ('umap_0' and 'umap_1'), the metadata, and the UMAP category.

    Notes
    -----
    This function assumes that metadata columns can be identified by the function 'infer_cp_features' with 'metadata=True'.
    """
    # Prepare UMAP input by removing metadata columns
    metadata_cols = infer_cp_features(data_df, metadata=True)
    metadata_df = data_df.loc[:, metadata_cols]

    if features == "infer":
        umap_data_df = data_df.drop(metadata_cols, axis="columns")
    else:
        umap_data_df = data_df.loc[:, features]

    # Apply UMAP
    reducer = umap.UMAP(random_state=123)
    embedding = reducer.fit_transform(umap_data_df)
    
    # Setup plotting logic
    embedding_df = pd.DataFrame(embedding, columns=['umap_0', 'umap_1'])
    embedding_df = embedding_df.merge(metadata_df, left_index=True, right_index=True)
    embedding_df = embedding_df.assign(Metadata_umap_category=umap_category)
    
    return embedding_df


def perform_clustering(
        df: pd.DataFrame,
        features: Union[str, List[str]],
        pca_n_components: int,
        class_column: str,
        positive_class: Union[int, str],
        feature_category: str,
        low_k: int, 
        high_k: int
        ) -> pd.DataFrame:
    """
    Perform KMeans clustering on a subset of a dataframe and calculate:
        1) Silhouette width of clustering solution
        2) Enrichment of a class within clusters

    Parameters
    ----------
    df : pd.DataFrame
        The input dataframe. Should include feature columns and a class column.
    features : list of str
        The names of the feature columns to use for clustering.
    pca_n_components : int
        The number of principal coponents to use for PCA transformation
    class_column : str
        The name of the class column.
    positive_class : int or str
        The value in the class column that represents the "positive" class.
    all_features : str
        A label to add to the results dataframe.
    low_k : int
        The lower bound for the range of k for KMeans clustering.
    high_k : int
        The upper bound for the range of k for KMeans clustering.

    Returns
    -------
    pd.DataFrame
        A dataframe with the results.
        Columns are 'k', 'silhouette_width', 'average_enrichment', 'average_p_value', and 'feature_category'.

    """
    # Prepare output dataframe
    results = pd.DataFrame(
        columns=[
            "k",
            "silhouette_width",
            "average_enrichment",
            "maximum_enrichment",
            "average_p_value",
            "feature_category"
        ]
    )

    # Subset dataframe
    if features == "infer":
        metadata_cols = infer_cp_features(df, metadata=True)
        subset_df = df.drop(metadata_cols, axis="columns")
    else:
        subset_df = df.loc[:, features]

    # Transform data into PCA space to account for differential feature numbers influencing distance metrics
    pca = PCA(n_components=pca_n_components)
    subset_df = pca.fit_transform(subset_df)

    # Get boolean series indicating whether each sample belongs to the positive class
    is_in_class = df.loc[:, class_column] == positive_class

    # Perform kmeans clustering and calculate metrics
    for k in range(low_k, high_k+1):
        # KMeans clustering
        kmeans = KMeans(n_clusters=k, random_state=0, n_init=20).fit(subset_df)

        # Calculate silhouette width
        silhouette_width = silhouette_score(subset_df, kmeans.labels_)

        # Calculate enrichment for each cluster
        enrichments = []
        p_values = []
        for cluster_id in np.unique(kmeans.labels_):
            # Get boolean series indicating whether each sample is in the current cluster
            is_in_cluster = kmeans.labels_ == cluster_id

            # Calculate contingency table
            table = np.zeros((2, 2))
            table[0, 0] = np.sum(is_in_class & is_in_cluster)   # in class and in cluster
            table[0, 1] = np.sum(is_in_class & ~is_in_cluster)  # in class and not in cluster
            table[1, 0] = np.sum(~is_in_class & is_in_cluster)  # not in class and in cluster
            table[1, 1] = np.sum(~is_in_class & ~is_in_cluster) # not in class and not in cluster

            # Calculate Fisher's exact test
            odds_ratio, p_value = fisher_exact(table)
            enrichments.append(odds_ratio)
            p_values.append(p_value)

        # Calculate average enrichment
        average_enrichment = sum(enrichments) / len(enrichments)
        maximum_enrichment = np.max(enrichments)
        average_p_value = sum(p_values) / len(p_values)

       # Append results
        results_row = pd.DataFrame({
            "k": [k],
            "silhouette_width": [silhouette_width],
            "average_enrichment": [average_enrichment],
            "maximum_enrichment": [maximum_enrichment],
            "average_p_value": [average_p_value],
            "feature_category": [feature_category]
        })
        results = pd.concat([results, results_row], ignore_index=True)

    return results.reset_index(drop=True)


## Define paths

In [4]:
data_dir = pathlib.Path("..", "2.describe-data", "data", "merged")
signature_dir = pathlib.Path("..", "3.resistance-signature")

profile_file = pathlib.Path(f"{data_dir}/all_merged_profiles_before_feature_selection.csv.gz")
profile_feature_selection_file = pathlib.Path(f"{data_dir}/all_merged_profiles.csv.gz")

bz_signature_file = pathlib.Path(f"{signature_dir}/results/signatures/signature_summary_bortezomib_signature.tsv.gz")

output_umap_file = pathlib.Path("results", "umap_feature_summary.tsv.gz")
output_cluster_file = pathlib.Path("results", "clustering_feature_summary.tsv.gz")


## Load profiles

In [5]:
profile_df = pd.read_csv(profile_file, low_memory=False)

print(profile_df.shape)
profile_df.head()

(3957, 3544)


,Metadata_Plate,Metadata_Well,Metadata_batch,Metadata_cell_count,Metadata_cell_density,Metadata_celltype_shorthand_from_plate_graph,Metadata_clone_number,Metadata_date,Metadata_plate_ID,Metadata_plate_filename,...,Nuclei_Texture_Variance_RNA_10_02,Nuclei_Texture_Variance_RNA_10_03,Nuclei_Texture_Variance_RNA_20_00,Nuclei_Texture_Variance_RNA_20_01,Nuclei_Texture_Variance_RNA_20_02,Nuclei_Texture_Variance_RNA_20_03,Nuclei_Texture_Variance_RNA_5_00,Nuclei_Texture_Variance_RNA_5_01,Nuclei_Texture_Variance_RNA_5_02,Nuclei_Texture_Variance_RNA_5_03
0,219905,B02,2021_03_03_Batch14,4269,2.5x10^3 cells/well,1.0,WT_parental,20210219.0,NaN,NaN,...,-1.007074,-1.010473,-1.018616,-1.038237,-1.005599,-1.015286,-1.020732,-1.009446,-1.019474,-1.012719
1,219905,B03,2021_03_03_Batch14,1688,2.5x10^3 cells/well,2.0,CloneA,20210219.0,NaN,NaN,...,-0.986309,-1.000248,-1.022190,-1.045947,-1.016479,-1.038013,-0.999631,-0.999447,-0.987759,-0.977462
2,219905,B04,2021_03_03_Batch14,2238,2.5x10^3 cells/well,3.0,CloneE,20210219.0,NaN,NaN,...,-1.371187,-1.358646,-1.353107,-1.355997,-1.357253,-1.352782,-1.360769,-1.364501,-1.366883,-1.360609
3,219905,B05,2021_03_03_Batch14,3358,2.5x10^3 cells/well,4.0,WT clone 01,20210219.0,NaN,NaN,...,-1.454741,-1.437857,-1.441065,-1.458476,-1.441738,-1.481096,-1.456043,-1.463368,-1.462129,-1.456112
4,219905,B06,2021_03_03_Batch14,291,2.5x10^3 cells/well,5.0,WT clone 02,20210219.0,NaN,NaN,...,0.334975,0.403553,0.461756,0.434181,0.315446,0.586550,0.342533,0.354969,0.379586,0.361450


In [6]:
# Load feature selected dataframe
profile_feature_select_df = pd.read_csv(profile_feature_selection_file)

print(profile_feature_select_df.shape)
profile_feature_select_df.head()

(3957, 458)


,Metadata_Plate,Metadata_Well,Metadata_batch,Metadata_cell_count,Metadata_cell_density,Metadata_celltype_shorthand_from_plate_graph,Metadata_clone_number,Metadata_date,Metadata_plate_map_name,Metadata_time_to_adhere,...,Nuclei_Texture_Correlation_RNA_20_00,Nuclei_Texture_Correlation_RNA_20_02,Nuclei_Texture_Entropy_DNA_20_03,Nuclei_Texture_Entropy_ER_20_03,Nuclei_Texture_InfoMeas1_DNA_5_02,Nuclei_Texture_InfoMeas1_ER_5_00,Nuclei_Texture_InfoMeas1_Mito_5_00,Nuclei_Texture_InfoMeas1_RNA_5_00,Nuclei_Texture_SumAverage_DNA_20_03,Nuclei_Texture_SumVariance_AGP_20_03
0,219905,B02,2021_03_03_Batch14,4269,2.5x10^3 cells/well,1.0,WT_parental,20210219.0,219905,48 hr,...,-0.393503,-0.656457,-1.525862,-0.689886,1.381230,1.683708,0.890482,1.819708,-0.654685,-1.201057
1,219905,B03,2021_03_03_Batch14,1688,2.5x10^3 cells/well,2.0,CloneA,20210219.0,219905,48 hr,...,1.202557,1.040639,-0.737619,-0.873918,0.133264,-0.753028,-0.729984,-0.761175,-1.688990,-0.894227
2,219905,B04,2021_03_03_Batch14,2238,2.5x10^3 cells/well,3.0,CloneE,20210219.0,219905,48 hr,...,1.782753,1.151987,-1.144326,-2.081855,0.613646,0.124588,0.677733,-0.322159,-1.776875,-1.199676
3,219905,B05,2021_03_03_Batch14,3358,2.5x10^3 cells/well,4.0,WT clone 01,20210219.0,219905,48 hr,...,-0.730221,-0.534683,-1.467039,-1.865624,1.895769,1.285182,0.735492,1.617357,-1.205825,-1.531649
4,219905,B06,2021_03_03_Batch14,291,2.5x10^3 cells/well,5.0,WT clone 02,20210219.0,219905,48 hr,...,0.727340,1.228002,1.874837,0.763804,-1.437746,-0.150566,-0.905429,-0.617372,1.571506,0.849895


In [7]:
# Load bortezomib signature features
bz_sig_df = pd.read_csv(bz_signature_file, sep="\t")

bz_sig_features = bz_sig_df.query("final_signature").features.to_list()

print(bz_sig_df.shape)
print(len(bz_sig_features))
bz_sig_df.head()

(782, 8)
45


,features,non_status_significant_exclude,batch_exclude,cell_count_exclude,non_specific_exclude,treatment_time_exclude,final_signature,dataset
0,Cells_AreaShape_Compactness,False,False,False,True,False,False,bortezomib
1,Cells_AreaShape_Eccentricity,True,False,True,True,False,False,bortezomib
2,Cells_AreaShape_Extent,False,False,False,True,False,False,bortezomib
3,Cells_AreaShape_FormFactor,False,False,True,True,False,False,bortezomib
4,Cells_AreaShape_MeanRadius,True,False,False,True,False,False,bortezomib


In [8]:
# Get all features except those in the signature
all_features = infer_cp_features(profile_df)
all_features_except_bz_sig_features = [x for x in all_features if x not in bz_sig_features]
len(all_features_except_bz_sig_features)

3483

In [9]:
# Get random features
random_features = random.sample(all_features_except_bz_sig_features, len(bz_sig_features))

## Step 1: Calculate UMAP coordinates using four different feature spaces:

In [10]:
# 1) All feature umap
umap_all_feature_df = process_umap(
    profile_df,
    umap_category="all_features"
)

# 2) Feature selected umap
umap_fs_feature_df = process_umap(
    profile_feature_select_df,
    umap_category="feature_selected"
)

# 3) Bortezomib signature features
umap_bz_sig_df = process_umap(
    profile_df,
    features=bz_sig_features,
    umap_category="bortezomib_signature_features"
)

# 4) All features except bortezomib signature
umap_non_bz_sig_df = process_umap(
    profile_df,
    features=all_features_except_bz_sig_features,
    umap_category="all_except_bortezomib_signature_features"
)

# 5) 45 random CellProfiler features
umap_random_sig_df = process_umap(
    profile_df,
    features=random_features,
    umap_category="random_45_features"
)

In [11]:
# Output umap summary
umap_df = pd.concat(
    [
        umap_all_feature_df,
        umap_fs_feature_df,
        umap_bz_sig_df,
        umap_non_bz_sig_df,
        umap_random_sig_df
    ]
).reset_index(drop=True)

umap_df.to_csv(output_umap_file, sep="\t", index=False)

print(umap_df.shape)
umap_df.head()

(19785, 19)


,umap_0,umap_1,Metadata_Plate,Metadata_Well,Metadata_batch,Metadata_cell_count,Metadata_cell_density,Metadata_celltype_shorthand_from_plate_graph,Metadata_clone_number,Metadata_date,Metadata_plate_ID,Metadata_plate_filename,Metadata_plate_map_name,Metadata_time_to_adhere,Metadata_treatment,Metadata_treatment_shorthand_from_plate_graph,Metadata_treatment_time,Metadata_clone_type,Metadata_umap_category
0,5.518796,3.745502,219905,B02,2021_03_03_Batch14,4269,2.5x10^3 cells/well,1.0,WT_parental,20210219.0,NaN,NaN,219905,48 hr,2.33 nM bortezomib,2.0,4 hr,wildtype,all_features
1,10.290759,2.276558,219905,B03,2021_03_03_Batch14,1688,2.5x10^3 cells/well,2.0,CloneA,20210219.0,NaN,NaN,219905,48 hr,2.33 nM bortezomib,2.0,4 hr,resistant,all_features
2,10.443436,1.720236,219905,B04,2021_03_03_Batch14,2238,2.5x10^3 cells/well,3.0,CloneE,20210219.0,NaN,NaN,219905,48 hr,2.33 nM bortezomib,2.0,4 hr,resistant,all_features
3,9.149369,0.148676,219905,B05,2021_03_03_Batch14,3358,2.5x10^3 cells/well,4.0,WT clone 01,20210219.0,NaN,NaN,219905,48 hr,2.33 nM bortezomib,2.0,4 hr,wildtype,all_features
4,9.998218,11.138648,219905,B06,2021_03_03_Batch14,291,2.5x10^3 cells/well,5.0,WT clone 02,20210219.0,NaN,NaN,219905,48 hr,2.33 nM bortezomib,2.0,4 hr,wildtype,all_features


## Step 2: Perform clustering and calculate enrichment and silhouette scores for the four feature spaces

In [12]:
low_k = 2
high_k = 14
pca_n_components = 30

# 1) All feature clustering
all_feature_cluster_df = perform_clustering(
    df = profile_df,
    features = "infer",
    pca_n_components=pca_n_components,
    class_column = "Metadata_clone_type",
    positive_class = "wildtype",
    feature_category = "all_features",
    low_k = low_k, 
    high_k = high_k
)

# 2) Feature selected clustering
fs_feature_cluster_df = perform_clustering(
    df = profile_feature_select_df,
    features = "infer",
    pca_n_components=pca_n_components,
    class_column = "Metadata_clone_type",
    positive_class = "wildtype",
    feature_category = "feature_selected",
    low_k = low_k, 
    high_k = high_k
)

# 3) Bortezomib signature features clustering
bz_feature_cluster_df = perform_clustering(
    df = profile_df,
    features = bz_sig_features,
    pca_n_components=pca_n_components,
    class_column = "Metadata_clone_type",
    positive_class = "wildtype",
    feature_category = "bortezomib_signature_features",
    low_k = low_k, 
    high_k = high_k
)

# 4) All features except bortezomib signature clustering
non_bz_feature_cluster_df = perform_clustering(
    df = profile_df,
    features = all_features_except_bz_sig_features,
    pca_n_components=pca_n_components,
    class_column = "Metadata_clone_type",
    positive_class = "wildtype",
    feature_category = "all_except_bortezomib_signature_features",
    low_k = low_k, 
    high_k = high_k
)

# 5) Random features clustering
random_feature_cluster_df = perform_clustering(
    df = profile_df,
    features = random_features,
    pca_n_components=pca_n_components,
    class_column = "Metadata_clone_type",
    positive_class = "wildtype",
    feature_category = "random_45_features",
    low_k = low_k, 
    high_k = high_k
)

In [13]:
# Output clustering summary
clustering_df = pd.concat(
    [
        all_feature_cluster_df,
        fs_feature_cluster_df,
        bz_feature_cluster_df,
        non_bz_feature_cluster_df,
        random_feature_cluster_df
    ]
).reset_index(drop=True)

clustering_df.to_csv(output_cluster_file, sep="\t", index=False)

print(clustering_df.shape)
clustering_df.head()

(65, 6)


,k,silhouette_width,average_enrichment,maximum_enrichment,average_p_value,feature_category
0,2,0.270829,1.170358,1.778420,3.112696e-19,all_features
1,3,0.192410,1.276070,2.594515,1.467894e-02,all_features
2,4,0.167391,1.400925,3.462636,1.548676e-01,all_features
3,5,0.137051,1.350022,3.679567,3.820717e-05,all_features
4,6,0.135188,1.395909,3.988519,6.906626e-06,all_features
